In [0]:
import pandas as pd
import numpy as np

In [4]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [0]:
import json # library to handle JSON files
import requests # library to handle requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

## Data cleaning

In [63]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.'
table = pd.read_html(url)
table

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 282      M8Z         Etobicoke              Mimico NW
 283      M8Z         Etobicoke     The Queensway West
 284      M8Z         Etobicoke  Royal York South West
 285      M8Z         Etobicoke         South of Bloor
 286      M9Z      Not assigned           Not assigned
 
 [287 rows x 3 columns],
                                                   0   ...   17
 0                                                NaN  ...  NaN
 1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...  ...  NaN
 2                                                 NL  ...   YT
 3                                                  A  ..

In [66]:
df = pd.DataFrame(table[0])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [67]:
df.columns = ['PostalCode','Borough','Neighborhood'] #rename the columns as required
#df = df[1:] #delete first row as it was the original titles
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [69]:
df.iloc[0]

PostalCode               M1A
Borough         Not assigned
Neighborhood    Not assigned
Name: 0, dtype: object

In [71]:
df=df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [79]:
#Now we combine the rows having same PostalCode and Borough
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()
df.index.size

103

In [81]:
df['PostalCode'].unique

<bound method Series.unique of 0      M1B
1      M1C
2      M1E
3      M1G
4      M1H
      ... 
98     M9N
99     M9P
100    M9R
101    M9V
102    M9W
Name: PostalCode, Length: 103, dtype: object>

In [0]:
df_size = df.index.size #get index size
n=0 #initialize n
for n in range(0,df_size): #iterate rows
    if df.iloc[n,2]=='Not assigned': #check if not assigned on Neighborhood
        print('Before ',df.iloc[n,2],'and should be',df.iloc[n,1])
        df.iloc[n,2]=df.iloc[n,1] #replace not assigned neighborhood with Borough
        print('Now ',df.iloc[n,2],'is',df.iloc[n,1])
        n=n+1

In [86]:
df.head(103)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [87]:
df.shape

(103, 3)

This finishes the Data Cleaning